<a href="https://colab.research.google.com/github/claytonturner358/final-project/blob/main/Ancient_Metagenomics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Install Kraken2 and related tools

In [ ]:
!apt-get update
!apt-get install -y kraken2

### 2. Download the minikraken2_v2_8GB_201904.tgz database to your Google Drive and then run the code below.
This application was tested using the minikraken2_v2_8GB_201904.tgz database.
The minikraken2_v2_8GB_201904.tgz database can be downloaded from the link below. Note: It is located near the bottom of the download page.

 https://github.com/BenLangmead/aws-indexes/blob/master/docs/k2.md


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create a directory for the database
!mkdir -p kraken_db

# Define the path to the database in Google Drive
GOOGLE_DRIVE_DB_PATH = '/content/drive/MyDrive/minikraken2_v2_8GB_201904.tgz'
LOCAL_DB_ARCHIVE = 'kraken_db/minikraken2_v2_8GB_201904.tgz'

# Copy the database from Google Drive to the local kraken_db directory
!cp "{GOOGLE_DRIVE_DB_PATH}" "{LOCAL_DB_ARCHIVE}"

# Extract the database
# The tar command extracts contents into 'kraken_db/'. If the archive has a top-level folder, it will be created inside kraken_db.
!tar -xzf "{LOCAL_DB_ARCHIVE}" -C kraken_db/

# Set the database path to the *actual* directory containing the .k2d files, as confirmed by 'ls -R kraken_db' output.
KRAKEN_DB_PATH = 'kraken_db/minikraken2_v2_8GB_201904_UPDATE'
print(f"Kraken2 database path set to: {KRAKEN_DB_PATH}")

In [ ]:
# This section of code is to ensure the proper  files were included with the kraken_db.
# Some downloads are missing the "taxo.k2d" file and this serves as a means of checking for it.

print("--- Contents of kraken_db directory ---")
!ls -R kraken_db

### 3. Upload your FASTA file

In [ ]:
import os
from google.colab import files

# Attempt to upload a FASTA file
uploaded = files.upload()

if uploaded:
  INPUT_FASTA_FILE = list(uploaded.keys())[0]
  print(f'User uploaded file "{INPUT_FASTA_FILE}" with length {len(uploaded[INPUT_FASTA_FILE])} bytes')
else:
  print("No file uploaded. Please upload a FASTA file to proceed.")
  INPUT_FASTA_FILE = '' # Set to empty to prevent running Kraken2 with an invalid path

print(f"Using '{INPUT_FASTA_FILE}' as the input FASTA file.")

# Display the first few lines of the input FASTA file to verify its content
if INPUT_FASTA_FILE and os.path.exists(INPUT_FASTA_FILE):
  print(f"\n--- First 5 lines of {INPUT_FASTA_FILE} ---")
  with open(INPUT_FASTA_FILE, 'r') as f:
    for i, line in enumerate(f):
      if i >= 5: break
      print(line.strip())
  print("---------------------------------------")
else:
  print(f"Error: Input FASTA file '{INPUT_FASTA_FILE}' does not exist or is empty.")

### 4. Run Kraken2 on your FASTA file

This command will classify your sequences. The output file (`sample.kraken`) will contain the classification for each read, and the report file (`sample.report`) will summarize the classifications.

In [ ]:
# Ensure INPUT_FASTA_FILE is the one uploaded by the user from cell 1693540a
KRAKEN_OUTPUT_FILE = 'sample.kraken'
KRAKEN_REPORT_FILE = 'sample.report'

!kraken2 --db "{KRAKEN_DB_PATH}" \
         --threads 4 \
         --output "{KRAKEN_OUTPUT_FILE}" \
         --report "{KRAKEN_REPORT_FILE}" \
         "{INPUT_FASTA_FILE}"

print(f"Kraken2 classification complete. Output: {KRAKEN_OUTPUT_FILE}, Report: {KRAKEN_REPORT_FILE}")

### 5. Display the Kraken report

You can view the generated report file directly.

In [ ]:
print(f"--- Contents of {KRAKEN_REPORT_FILE} ---")
!cat "{KRAKEN_REPORT_FILE}"

### 6. Format the report with column headers and a taxonomic key

In [ ]:
# Define the output file name
OUTPUT_FORMATTED_REPORT_FILE = 'formatted_report.txt'

# Key for taxonomic levels
taxonomic_level_key = [
    "U: unclassified",
    "R: root",
    "D: domain",
    "P: phylum",
    "C: class",
    "O: order",
    "F: family",
    "G: genus",
    "S: species",
    "other single letters may represent subspecies, strain, or other finer distinctions"
]

# Column headers for the report
column_headers = [
    "Percentage",
    "Reads_Assigned_To_Clade",
    "Reads_Assigned_Directly",
    "Taxonomic_Level",
    "Taxon_ID",
    "Taxon_Name"
]

print(f"Formatting Kraken report and saving to {OUTPUT_FORMATTED_REPORT_FILE}...")

with open(KRAKEN_REPORT_FILE, 'r') as infile, open(OUTPUT_FORMATTED_REPORT_FILE, 'w') as outfile:
    # Write the key explanation
    outfile.write("### Kraken Report Taxonomic Level Key ###\n")
    for entry in taxonomic_level_key:
        outfile.write(f"# {entry}\n")
    outfile.write("#####################################\n\n")

    # Write the column headers
    outfile.write("\t".join(column_headers) + "\n")

    # Copy the content of the original report
    for line in infile:
        # Original Kraken report lines are tab-separated, so we can just write them directly
        # after the header.
        outfile.write(line)

print(f"Formatted report saved to: {OUTPUT_FORMATTED_REPORT_FILE}")


In [ ]:
# Display the first few lines of the formatted report to verify
print(f"--- First 35 lines of {OUTPUT_FORMATTED_REPORT_FILE} ---")
with open(OUTPUT_FORMATTED_REPORT_FILE, 'r') as f:
    for i, line in enumerate(f):
        if i >= 35: break
        print(line.strip())
print("-----------------------------------------------------")
